# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
import pandas as pd
import numpy as np
import string
from nltk import pos_tag, ne_chunk
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table('clean_messages', engine) 
df = df[df['related-1']!=2]
X = df['message']
y = df.drop(['id','message','original','genre'],axis=1)

In [3]:
X.shape, y.shape

((26028,), (26028, 36))

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    #remove punctuation characters
    text = text.translate(str.maketrans('', '', string.punctuation))
    #lemmatize, convert to lowercase, remove leading/trailing white space
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text).lower().strip()
    #tokenize
    words = word_tokenize(text)
    #stop words removal
    words = [w for w in words if w not in stopwords.words("english")]
    #Stemming
    words = [PorterStemmer().stem(w) for w in words]
    clean_tokens = []
    for tok in words:
        clean_tokens.append(tok)
    return clean_tokens

print(tokenize(X[0]))
print(tokenize(X[1]))
print(tokenize(X[26203]))

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']
['hurrican']
['bangkok', '24', 'januari', '2012', 'nnt', 'prime', 'minist', 'yingluck', 'shinawatra', 'attend', 'meet', 'perman', 'secretari', 'variou', 'ministri', 'urg', 'quickli', 'distribut', 'flood', 'compens', 'wise', 'util', 'budget']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('clf',MultiOutputClassifier(RandomForestClassifier())),
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
#train
pipeline.fit(X_train,y_train)
#predict on test data
y_pred = pipeline.predict(X_test)

In [7]:
print(y_pred)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 1]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]


In [8]:
y_pred_columns = y_test.columns
y_pred = pd.DataFrame(y_pred, columns = y_pred_columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
target_names = y_test.columns
print(classification_report(y_test, y_pred, target_names=target_names))

                          precision    recall  f1-score   support

               related-1       0.85      0.91      0.88      4995
               request-0       0.81      0.42      0.56      1143
                 offer-0       0.00      0.00      0.00        29
           aid_related-0       0.74      0.60      0.67      2742
          medical_help-0       0.52      0.06      0.11       552
      medical_products-0       0.65      0.13      0.22       313
     search_and_rescue-0       0.65      0.07      0.12       169
              security-0       0.33      0.01      0.02       113
              military-0       0.74      0.09      0.15       198
           child_alone-0       0.00      0.00      0.00         0
                 water-0       0.79      0.23      0.36       425
                  food-0       0.85      0.41      0.56       743
               shelter-0       0.79      0.37      0.50       592
              clothing-0       0.71      0.11      0.19        91
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc4fdd7ec80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
parameters = {'clf__estimator__max_depth': [10],
              'clf__estimator__min_samples_leaf':[2, 5],
             'clf__estimator__min_samples_split': [2, 4],}

cv = GridSearchCV(pipeline, parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10], 'clf__estimator__min_samples_leaf': [2, 5], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 2)
np.random.seed(42)
y_pred2 = cv.fit(X_train, y_train)
prediction2 = y_pred2.predict(X_test)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf_

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 37.7min finished


In [13]:
print(classification_report(y_test, prediction2 , target_names = target_names))

                          precision    recall  f1-score   support

               related-1       0.77      1.00      0.87      4995
               request-0       0.00      0.00      0.00      1143
                 offer-0       0.00      0.00      0.00        29
           aid_related-0       0.88      0.11      0.20      2742
          medical_help-0       0.00      0.00      0.00       552
      medical_products-0       0.00      0.00      0.00       313
     search_and_rescue-0       0.00      0.00      0.00       169
              security-0       0.00      0.00      0.00       113
              military-0       0.00      0.00      0.00       198
           child_alone-0       0.00      0.00      0.00         0
                 water-0       0.00      0.00      0.00       425
                  food-0       0.00      0.00      0.00       743
               shelter-0       0.00      0.00      0.00       592
              clothing-0       0.00      0.00      0.00        91
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
import pandas as pd
import numpy as np
import string
from nltk import pos_tag, ne_chunk
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle


def load_data(database_filepath):

    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table('DisasterResponse', engine) 
    X = df['message']
    y = df.drop(['id','message','original','genre'],axis=1)
    category_names = y.columns
    
    return X, y, category_names


def tokenize(text):
    #remove punctuation characters
    text = text.translate(str.maketrans('', '', string.punctuation))
    #lemmatize, convert to lowercase, remove leading/trailing white space
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text).lower().strip()
    #tokenize
    words = word_tokenize(text)
    #stop words removal
    words = [w for w in words if w not in stopwords.words("english")]
    #Stemming
    words = [PorterStemmer().stem(w) for w in words]
    clean_tokens = []
    for tok in words:
        clean_tokens.append(tok)
    return clean_tokens


def build_model():
    pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('clf',MultiOutputClassifier(RandomForestClassifier())),
                    ])
    
    return pipeline

def evaluate_model(model, X_test, Y_test, category_names):
    
    #predict on test data
    y_pred = model.predict(X_test)
    
    print(classification_report(Y_test, y_pred, target_names=category_names))


def save_model(model, model_filepath):
    
    pickle.dump(model,open(model_filepath,'wb'))


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()